Készítsen ImageDataGeneratort, amely a megadott források közül a train könyvtárban levő tanítómintákat
fogja a modellnek átadni.
a. A tanítóadatokként használt képeket 64×64-es méretű színes képekre méretezze át, 64-es méretű
kötegekben adja át a hálózatnak
b. Készítsen a validációs és tesztadatokhoz is generátort!

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1.0/255,  
    featurewise_std_normalization=True  
)

train_generator = train_datagen.flow_from_directory(
    'path/to/train',  
    target_size=(64, 64),  
    batch_size=64,  
    class_mode='categorical' 
)

validation_datagen = ImageDataGenerator(
    rescale=1.0/255,  
    featurewise_std_normalization=True  
)

validation_generator = validation_datagen.flow_from_directory(
    'path/to/validation',  
    target_size=(64, 64),  
    batch_size=64,  
    class_mode='categorical'  
)

test_datagen = ImageDataGenerator(
    rescale=1.0/255,  
    featurewise_std_normalization=True  
)

test_generator = test_datagen.flow_from_directory(
    'path/to/test', 
    target_size=(64, 64), 
    batch_size=64,
    class_mode='categorical'
)

Készítsen modellt, amelynek a bemenete egy 64×64×3-as tenzor, rétegei pedig a lentebbiek szerint adottak!

a. Az első réteg 32 darab 5×5 méretű kétdimenziós konvolúciós filtert tanítson;
b. E réteget regularizálja dropout technikával, 0.2-es értékű dropout rátával;
c. A második réteg 64 darab 5×5 méretű kétdimenziós konvolúciós filtert tanítson;
d. E réteget is regularizálja dropout technikával, 0.2-es értékű dropout rátával;
e. A harmadik réteg kétdimenziós maxpooling legyen, tanított paraméter nélkül, 2×2-es méretű ablakkal;
f. Ezt követően összesen négy darab kétdimenziós konvolúciósfiltert következik egymás után, mindegyik
64 darab 3×3 méretű kernellel, melyek mindegyike 0.1 értékű dropout rátával adott dropout
regularizációt alkalmazzon;
g. Ezt követően egy újabb kétdimenziós maxpooling következik, tanított paraméter nélkül, 2×2-es
méretű ablakkal;
h. Végül ismét 3 egyforma kétdimenziós konvolúciós szűrő következik egymást követően: mindegyik 128
darab 3×3 méretű kernellel, és mindegyik 0.1 értékű dropouttal regularizálva;
i. Egy utolsó kétdimenziós maxpooling 2×2 méretű ablakkal;
j. Majd az eddigi kimenet aktivációs térképét (amelynek mérete itt 128×2×2 kell legyen) alakítsuk át egy
egydimenziós 512 elemű vektorrá „lapítással”;
k. E vektor minden elemét pedig kössük rá egy 128 neuronból álló réteg minden elemére;
l. Majd végül egy 8 elemű réteg legyen a h
zat kimenete.
3. Minden neuron aktivációja ReLU legyen, kivéve a kimeneti réteget, ahol softmaxot alkalmazzon!

In [ ]:
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, 

model = Sequential()
model.add(Conv2D(32, (5, 5), activation='relu', input_shape=(64, 64, 3)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(Dropout(0.2))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Dropout(0.1))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Dropout(0.1))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Dropout(0.1))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Dropout(0.1))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Dropout(0.1))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Dropout(0.1))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Dropout(0.1))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))

model.add(Dense(8, activation='softmax'))

Építse fel a modellt

a. Veszteségfüggvénynek olyat válasszon, amely alkalmas softmax alapú többosztályú klasszifikációra!
b. A választott optimalizációs módszer legyen adam;
c. A kiértékeléskor használt metrika a klasszifikációs pontosság legye

5. Jelenítse meg a modell strukturáját és a tanítható paraméterek számát!

In [ ]:
model.compile(
    optimizer='adam',  # Optimalizáló
    loss='categorical_crossentropy',  # Többosztályú klasszifikációs veszteségfüggvény
    metrics=['accuracy']  # Pontosság mint metrika
)

model.summary()

Tanítsa fel a neurális hálózatot a tanítóadatokon!

a. A tanítás során a tanítómintákat legfeljebb 1000 alkalommal adja át a hálózatnak;
b. Azonban ha a validációs adatokon mért veszteség nem javul 5 epochon át, akkor a tanítást állítsa le.

In [ ]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',  # A validációs veszteséget figyeljük
    patience=5,  # Ha 5 epochon keresztül nem javul, leállítjuk a tanítást
    restore_best_weights=True  # A legjobb súlyokat állítja vissza
)

history = model.fit(
    train_generator,  # Tanítóadatok
    epochs=1000,  # Maximum epoch
    validation_data=validation_generator,  # Validációs adatok
    callbacks=[early_stopping]  # Callback használata
)

Rajzolja ki a matplotlib segítségével a tanítás során a veszteség és a klasszifikációs pontosság változását!

In [ ]:
import matplotlib.pyplot as plt

# Tanítási és validációs veszteség ábrázolása
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)  # Az első ábra (1 sor, 2 oszlop, 1. hely)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss Over Epochs')
plt.legend()

# Tanítási és validációs pontosság ábrázolása
plt.subplot(1, 2, 2)  # Az második ábra (1 sor, 2 oszlop, 2. hely)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy Over Epochs')
plt.legend()

plt.tight_layout()
plt.show()

Írja ki az utolsó lépésben mért klasszifikációs pontosságot a tanítóadatokon és a validációs adatokon!
a. Értékelje ki a modell teljesítményét a tesztadatokon!

In [ ]:
# Utolsó epoch pontosság kiírása
train_accuracy = history.history['accuracy'][-1]
val_accuracy = history.history['val_accuracy'][-1]

print(f"Train Accuracy (last epoch): {train_accuracy:.2f}")
print(f"Validation Accuracy (last epoch): {val_accuracy:.2f}")

# Modell teljesítményének kiértékelése tesztadatokon
test_loss, test_accuracy = model.evaluate(test_generator)

print(f"Test Loss: {test_loss:.2f}")
print(f"Test Accuracy: {test_accuracy:.2f}")